In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file = 'south_park.txt'

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding="ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 359421 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])
text = text.replace('\n\n','\n')
text = text.replace('\r','')
text = text.replace('\t','')

At the bus stop.
Boys
School day, school day, teacher's golden ru...
Kyle Broflovski
Ah, damn it! My little brother's trying to follow me to school again.
Ike Broflovski
Zeeponanner.
Kyle
Ike, you can't come to school with me. [Ike Chortles.]


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

87 unique characters


In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
#Unique Characters with mapping
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '$' :   4,
  '%' :   5,
  '&' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '+' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'At the bus st' ---- characters mapped to int ---- > [27 74  1 74 62 59  1 56 75 73  1 73 74]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

A
t
 
t
h


In [10]:
#See sequences made for training
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"At the bus stop.\nBoys\nSchool day, school day, teacher's golden ru...\nKyle Broflovski\nAh, damn it! My "
"little brother's trying to follow me to school again.\nIke Broflovski\nZeeponanner.\nKyle\nIke, you can't"
" come to school with me. [Ike Chortles.]\nEric Cartman\nYeah, go home you little dildo!\nKyle\nDude, don'"
"t call my brother a dildo!\nStan Marsh\nWhat's a dildo?\nKyle\nWell, I don't know...[He faces Cartman and"
" points at him.] and I'll bet Cartman doesn't know either!\nCartman\nI know what it means!\nKyle\nWell, w"


In [11]:
#create the input and target chunks of text sequences
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
#See sample sequence input and target
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "At the bus stop.\nBoys\nSchool day, school day, teacher's golden ru...\nKyle Broflovski\nAh, damn it! My"
Target data: "t the bus stop.\nBoys\nSchool day, school day, teacher's golden ru...\nKyle Broflovski\nAh, damn it! My "


In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 1024

In [15]:
#create model architecture
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [16]:
#build the model
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [17]:
#separate batches for training
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 87) # (batch_size, sequence_length, vocab_size)


In [18]:
#See model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           44544     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          6295552   
_________________________________________________________________
dense (Dense)                (64, None, 87)            89175     
Total params: 6,429,271
Trainable params: 6,429,271
Non-trainable params: 0
_________________________________________________________________


In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([68, 74, 11, 78, 16, 74, 37,  0, 41, 24, 38, 55, 43, 61, 37, 82, 68,
       14, 74, 20, 47, 85, 16, 71, 67, 74, 45, 66, 80, 49, 15, 74, 56, 85,
       72, 71, 21, 40, 80, 85, 26,  2, 10, 48, 16, 16,  9, 26, 35, 28, 82,
       38, 66,  1, 56,  0, 25, 81, 78,  7, 22, 63, 14, 56, 62, 35, 17, 34,
       49,  0,  5, 67, 70, 66, 65, 36, 37, 62, 75, 76, 58, 50, 40, 82, 13,
       37, 72, 43, 29, 50, 50, 79, 36, 71, 30, 44, 47, 26, 49, 49],
      dtype=int64)

In [20]:
#create loss function
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 87)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.467412


In [21]:
#compile the model with the optimizer and loss function
model.compile(optimizer='adam', loss=loss)

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [23]:
#set number of epochs
EPOCHS=150

In [24]:
#train the model
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/150
53/53 [==============================] - 11s 198ms/step - loss: 3.2043
Epoch 2/150
53/53 [==============================] - 9s 162ms/step - loss: 2.4896
Epoch 3/150
53/53 [==============================] - 9s 162ms/step - loss: 2.1973
Epoch 4/150
53/53 [==============================] - 9s 168ms/step - loss: 1.9945
Epoch 5/150
53/53 [==============================] - 9s 164ms/step - loss: 1.8403
Epoch 6/150
53/53 [==============================] - 9s 165ms/step - loss: 1.7146
Epoch 7/150
53/53 [==============================] - 9s 166ms/step - loss: 1.6136
Epoch 8/150
53/53 [==============================] - 9s 167ms/step - loss: 1.5330
Epoch 9/150
53/53 [==============================] - 9s 167ms/step - loss: 1.4617
Epoch 10/150
53/53 [==============================] - 9s 168ms/step - loss: 1.4027
Epoch 11/150
53/53 [==============================] - 9s 165ms/step - loss: 1.3466
Epoch 12/150
53/53 [==============================] - 9s 167ms/step - loss: 1.2977
Epoch 13/150

53/53 [==============================] - 9s 170ms/step - loss: 0.1858
Epoch 100/150
53/53 [==============================] - 9s 169ms/step - loss: 0.1840
Epoch 101/150
53/53 [==============================] - 9s 170ms/step - loss: 0.1827
Epoch 102/150
53/53 [==============================] - 9s 167ms/step - loss: 0.1820
Epoch 103/150
53/53 [==============================] - 9s 170ms/step - loss: 0.1804
Epoch 104/150
53/53 [==============================] - 9s 166ms/step - loss: 0.1812
Epoch 105/150
53/53 [==============================] - 9s 170ms/step - loss: 0.1803
Epoch 106/150
53/53 [==============================] - 9s 170ms/step - loss: 0.1797
Epoch 107/150
53/53 [==============================] - 9s 169ms/step - loss: 0.1799
Epoch 108/150
53/53 [==============================] - 9s 169ms/step - loss: 0.1782
Epoch 109/150
53/53 [==============================] - 9s 167ms/step - loss: 0.1792
Epoch 110/150
53/53 [==============================] - 9s 170ms/step - loss: 0.1773
Epoch 

In [25]:
#load latest checkpoint of trained model
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_150'

In [26]:
#build the loaded trained model
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [27]:
#see summary of loaded model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            44544     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           6295552   
_________________________________________________________________
dense_1 (Dense)              (1, None, 87)             89175     
Total params: 6,429,271
Trainable params: 6,429,271
Non-trainable params: 0
_________________________________________________________________


In [34]:
#create mapping of output from lstm to corresponding character
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 2000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.3

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [36]:
print(generate_text(model, start_string="Wendy\n"))

Wendy
KYou guys, 'Cause y, sits on either, git is make condo in gla dicatea Lee
A sicke shouldn't be making.
Chef
How's it go and smile.
UTheyle
Yeah, what's care. It...hef humor!
Kyle
But the zombies are coming!
Stan
We'll hold 'em off.eHallowing day.
Mephesto
Beautiful, it's absolutely butt. [To himself.] Uhyouyg to scare me. Pery Gook and two hands reach out to reveal the outhirg Stan!
Kyle
Why's your boy Victoria
Are cool, l.
Liane
How are you doing?
Cartman
Well, I'm pissed off!
Liane
How brison' Marvin.
Kyle. Huthey him.
Stan
Yeah, yeah, we have natural gay homosey for the watch.
Cartman
Hey, that kind of looks like... Tom Selleck. He steps up but I know that show sh.
Wendy
Aaaah!
Kyle returns his back to Pip, which fires on tek Faruse.
Cartman
Looks like they have t though you for some reason find?
Kyle
There's no new kid, but the children are in gonna be a tree-hugger.
Cartman
Yeah hippie, go back to homo.

The bus pulks up.
Ms. Crabtree
SIT DOWN KID!!!
Cartman
[Hushs.] You gue